In [12]:
import os
import re
from functools import partial
from itertools import product
from typing import Iterable, List, Union

import multiprocess as mp
import pandas as pd
import tqdm

from graph_state.graph_state import *

In [ ]:
def print_statistics_for_parallelized_experiments(results):
    print("\n--- Experiment Run Summary ---")
    saved_count = sum(1 for r in results if r.startswith("Saved"))
    skipped_count = sum(1 for r in results if r.startswith("Skipped"))
    failed_count = sum(1 for r in results if r.startswith("Failed"))
    print(f"Successfully saved: {saved_count}")
    print(f"Skipped (existed):  {skipped_count}")
    print(f"Failed:             {failed_count}")
    if failed_count > 0:
        print("\nFailures:")
        for r in results:
            if r.startswith("Failed"):
                print(f"  - {r}")

In [ ]:
def _run_bell_sampling_worker(combination: tuple, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    """
    A single-process worker function for bell_sampling_fidelity_experiment.
    'combination' is a tuple: (g: GraphState, err: error model (str), fidelity: float, shots: int, stab_factor: string indicating how many stabilizer elements we want to pick)
    The process will run and save the data to the output_dir.

    Returns:
        A status string for logging.
    """
    g, err, fid, shots, stab_factor = combination

    if stab_factor == '2n':
        numstab = g.n * 2
    elif stab_factor == 'n^2':
        numstab = g.n ** 2
    elif stab_factor.isdigit():
        numstab = int(stab_factor)
    else:
        return f"Failed (param error): Stabilizer factor '{stab_factor}' not recognized"

    # Construct filename and check for overwrite
    filename = f"bell_fidelity_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}_numstab_{numstab}.npy"
    filepath = os.path.join(output_dir, filename)
    
    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"
    
    # Run the actual experiment
    try:
        # all_fidelities = [fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)]
        all_fidelities = []
        for seed_i in range(num_repeats):
            samples = bell_sampling(g, err, fid, shots, seed=seed_i)
            est_fidelity = fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)
            all_fidelities.append(est_fidelity)
            
        # Save the result
        np.save(filepath, np.array(all_fidelities))
        return f"Saved ({len(all_fidelities)} repeats): {filename}"
    except Exception as e:
        return f"Failed (runtime error): {filename} with error: {e}"

def bell_sampling_fidelity_experiment(
    graphs: Union[GraphState, List[GraphState]],
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    stabilizer_factors: Union[str, List[str]],
    output_dir: str,
    overwrite: bool = False
):
    """
    Runs a Bell sampling fidelity experiment in parallel for all combinations of parameters. 
    Each combination is run in a separate process.
    Arguments can be passed in as either a single instance or as a list and will be expanded.
    """
    # Normalize all inputs to be lists
    graphs_list = [graphs] if isinstance(graphs, GraphState) else list(graphs)
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)
    stabilizer_factors_list = [stabilizer_factors] if isinstance(stabilizer_factors, str) else list(stabilizer_factors)

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(graphs_list, err_models, fidelities, shots_list, stabilizer_factors_list))
    
    print(f"Starting Bell random sampling experiment for {len(graphs_list)} graph(s), "
          f"{len(err_models)} error model(s), {len(fidelities)} fidelity value(s), "
          f"{len(shots_list)} shot setting(s), and {len(stabilizer_factors_list)} stabilizer factor(s).")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_worker,
        num_repeats=num_repeats,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [ ]:
def _run_tomo_worker(combination: tuple, g: GraphState, overlap_observables: bool, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    """
    A single-process worker function for partial_tomo_experiment.
    'combination' is a tuple: (err, fid, total_shots)
    
    Returns:
        A status string for logging.
    """
    try:
        err, fid, total_shots = combination
        N = 2 ** g.n

        filename = f"tomo_{g.n}q_F{fid:.3f}_err_{err}_shots_{total_shots}.npy"
        filepath = os.path.join(output_dir, filename)

        if not overwrite and os.path.exists(filepath):
            return f"Skipped (exists): {filename}"

        all_diags_for_combo = []
        for seed_i in range(num_repeats):
            exps = dge_combined(g, err, fid, total_shots, overlap_observables=overlap_observables, seed=seed_i)
            exps = np.maximum(0, exps)
            diags = get_diagonals_from_all_stabilizer_observables(g, exps)
            all_diags_for_combo.append(diags)
            
        # Stack the results into a single 2D NumPy array
        stacked_diags = np.array(all_diags_for_combo) # shape (num_repeats, number_of_diagonals)

        # 5. Save the result
        np.save(filepath, stacked_diags)
        return f"Saved ({stacked_diags.shape}): {filename}"
    
    except Exception as e:
        # Provide more context in the error message
        return f"Failed: {filename} with error: {e}"

def partial_tomo_experiment_parallelized(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    total_shots: Union[int, Iterable[int]],
    overlap_observables: bool,
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    """
    Runs a partial tomography experiment in parallel for all combinations
    of parameters. Each combination is run in a separate process.

    Args:
        g (GraphState): The graph state object.
        err_model (Union[str, List[str]]): A single error model string or a list of them.
        fidelity (Union[float, Iterable[float]]): A single fidelity value or an iterable.
        total_shots (Union[int, Iterable[int]]): A single total_shot (count) value, 
                                                 or an iterable of values.
        num_repeats (int): The number of times to repeat the experiment
                           for each parameter combination.
        output_dir (str): The directory where the output .npy files will be saved.
        overwrite (bool): If False, skips the calculation if the output file
                          already exists. If True, it will always run and overwrite
                          any existing file.
    """
    # Normalize all inputs to be lists
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [total_shots] if isinstance(total_shots, int) else list(total_shots)

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting tomography experiment for {g.n} qubits ({len(err_models)} error model(s), "
          f"{len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_tomo_worker,
        g=g,
        overlap_observables=overlap_observables,
        num_repeats=num_repeats,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [ ]:
def _run_bell_sampling_diagonal_worker(combination: tuple, g: GraphState, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    # Unpack the combination tuple
    err, fid, shots = combination

    # Construct filename and check for overwrite
    filename = f"bell_diag_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}.npy"
    filepath = os.path.join(output_dir, filename)

    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"

    actual_shots = (shots + 1) // 2
    bell_samples = bell_sampling(g, err, fid, actual_shots * num_repeats, seed=num_repeats)
    samples_split = np.split(bell_samples, num_repeats)

    all_diags_for_combo = []

    for samples in samples_split:
        exps = np.array(
            [
                expectation_value_of_observables_from_bell_bitpacked(
                    np.packbits(stab, bitorder="little"), samples
                )
                for stab in g.generate_all_int_staiblizers()
            ]
        )
        sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
        diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)
        all_diags_for_combo.append(diags)

    stacked_diags = np.array(all_diags_for_combo) # Shape (num_repeats, number_of_diagonals)
    np.save(filepath, stacked_diags)
    return f"Saved ({stacked_diags.shape}): {filename}"

def bell_sampling_diagonal_experiment(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)

    # Create the directory for saving results if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting Bell diagonal experiment for {g.n} qubits, {len(err_models)} error model(s), {len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    print(f"Starting experiment for {len(combinations)} parameter combinations...")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_diagonal_worker,
        num_repeats=num_repeats,
        g=g,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [ ]:
def _run_bell_sampling_diagonal_single_run_worker(combination: tuple[GraphState, str, float, int, int], output_dir: str, overwrite: bool) -> str:
    g, err, fid, shots, repeat_idx = combination

    # Construct filename and check for overwrite
    filename = f"bell_scalability_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}_repeat_{repeat_idx}.npy"
    filepath = os.path.join(output_dir, filename)

    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"

    actual_shots = (shots + 1) // 2
    bell_samples = bell_sampling(g, err, fid, actual_shots, seed=1000 * g.n + repeat_idx)

    exps = np.array(
        [
            expectation_value_of_observables_from_bell_bitpacked(
                np.packbits(stab, bitorder="little"), bell_samples
            )
            for stab in g.generate_all_int_staiblizers()
        ]
    )
    sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
    diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)

    np.save(filepath, diags)
    return f"Saved ({diags.shape}): {filename}"

def bell_diagonal_scalability_experiment(
    graphs: Union[GraphState, List[GraphState]],
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    # Normalize all inputs to be lists
    graphs_list = [graphs] if isinstance(graphs, GraphState) else list(graphs)
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)
    repeat_indices = [i for i in range(num_repeats)]

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(graphs_list, err_models, fidelities, shots_list, repeat_indices))
    
    print(f"Starting Bell random sampling experiment for {len(graphs_list)} graph(s), "
          f"{len(err_models)} error model(s), {len(fidelities)} fidelity value(s), "
          f"{len(shots_list)} shot setting(s), and {num_repeats} trials.")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_diagonal_single_run_worker,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [ ]:
"""
We generate the data for 8 qubit graph here.
    1. vary from F=0.5 to F=1 with N = 10^4
    2. vary from 10^2 to 10^5 with F = 0.9 (plotting later)
    X-axis will be log scale so we need to do log scaling X-axis (N)
"""

g = GraphState(8, "complete")
shots = 10_000
err_model = "single-qubit-dephasing"
# err_model = "depolarizing"
repeat = 1000
OVERWRITE = False

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="data-new/dephasing/dge_group_nonoverlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="data-new/dephasing/dge_group_nonoverlapped",
    overwrite=False,
)

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="data-new/dephasing/dge_group_overlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="data-new/dephasing/dge_group_overlapped",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=0.9,
    num_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    num_repeats=repeat,
    output_dir="data-new/dephasing/bsqn",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    num_shots=shots,
    num_repeats=repeat,
    output_dir="data-new/dephasing/bsqn",
    overwrite=False,
)

# scalability experiment
for n in range(2, 16):
    g = GraphState(n, "complete")
    partial_tomo_experiment_parallelized(
        g,
        'depolarizing',
        0.9,
        2 * 10_000,
        overlap_observables=True,
        num_repeats=25,
        output_dir="data-new/scalability/dge_group_overlapped",
        overwrite=False,
    )
bell_diagonal_scalability_experiment(
    [GraphState(n, "complete") for n in range(2, 24)],
    'depolarizing',
    fidelity=0.9,
    num_shots=20_000,
    num_repeats=25,
    output_dir="data-new/scalability/bsqn",
    overwrite=False,
)

## Now we process the raw data into CSV files keeping only the relevant information for plotting and archivals

In [9]:
def calculate_metrics_from_diags(diags_array: np.ndarray, true_diags: np.ndarray):
    """
    Calculates various metrics from diagonal measurements for each repeat.
    
    Args:
        diags_array: A 2D array of shape (num_repeats, num_diagonals).
        true_diags: A 1D array of the true diagonal values.
        input_fidelity: The fidelity value used as input for this experiment.

    Returns:
        A tuple containing three 1D arrays (one value per repeat):
        - delta_a_norm_2: The L2 norm of the difference vector.
        - estimated_fidelities: The calculated fidelity for each repeat.
        - delta_fidelities: The difference between input and estimated fidelity.
    """

    original_true_diags = np.array(true_diags)
    true_diags = true_diags[np.newaxis, :]

    # L2 norm of the difference vector (delta_a) for each repeat
    delta_a_norm_2 = np.linalg.norm(diags_array - true_diags, axis=1)
    delta_a_norm_1 = np.linalg.norm(diags_array - true_diags, ord=1, axis=1)
    
    estimated_fidelities = diags_array[:, 0]
    delta_fidelities = np.abs(original_true_diags[0] - estimated_fidelities)
    
    return delta_a_norm_2, delta_fidelities, delta_a_norm_1

def load_multiruns_diagonal_data(output_dir: str, file_prefix: str, df_prefix = None):
    """Loads all .npy files, calculates metrics, and returns pandas DataFrames."""
    all_data = []
    
    file_pattern = re.compile(
        rf"{file_prefix}_(\d+)q_F([\d.]+)_err_(.*?)_shots_(\d+).npy"
    )

    for filename in os.listdir(output_dir):
        match = file_pattern.match(filename)
        if match:
            qubits, F_str, err, shots = match.groups()
            qubits = int(qubits)
            input_F = float(F_str)
            
            filepath = os.path.join(output_dir, filename)
            diags_data = np.load(filepath)
            true_diags = get_true_diagonals(qubits, input_F, err)
            
            # Calculate all metrics for the loaded data
            delta_norms_2, delta_fidelities, delta_norms_1 = calculate_metrics_from_diags(diags_data, true_diags)

            # testing the second element
            estimated_second_element = diags_data[:, 1]
            
            # Delta b=1
            delta_b_1 = np.abs(np.array([true_diags[1] for _ in estimated_second_element]) - estimated_second_element)
            
            # Append one record per repeat
            for i in range(len(delta_norms_2)):
                all_data.append({
                    "prefix": file_prefix if df_prefix is None else df_prefix,
                    "qubits": int(qubits),
                    "input_fidelity": input_F,
                    "error_model": err,
                    "total_shots": int(shots),
                    "repeats": i,
                    "diag_sanity": np.sum(diags_data[i]), # sums the vector; it should sum to 1
                    "vector_a_norm_1": np.linalg.norm(diags_data[i], ord=1),
                    "vector_a_norm_2": np.linalg.norm(diags_data[i]),
                    "delta_a_norm_1": delta_norms_1[i],
                    "delta_a_norm_2": delta_norms_2[i],
                    "est_fidelity": diags_data[i][0],
                    "delta_fidelity": delta_fidelities[i],
                    "delta_b=1": delta_b_1[i],
                })
                
    if not all_data:
        return pd.DataFrame(), pd.DataFrame()
        
    df = pd.DataFrame(all_data)
    
    return df

def load_scalability_data(output_dir: str, file_prefix: str, df_prefix: str = None):
    """Loads all .npy files, calculates metrics, and returns pandas DataFrames."""
    all_data = []
    
    file_pattern = re.compile(
        rf"{file_prefix}_(\d+)q_F([\d.]+)_err_(.*?)_shots_(\d+)_repeat_(\d+).npy"
    )

    for filename in os.listdir(output_dir):
        match = file_pattern.match(filename)
        if match:
            qubits, F_str, err, shots, repeat_idx = match.groups()
            qubits = int(qubits)
            input_F = float(F_str)
            
            filepath = os.path.join(output_dir, filename)
            diags_data_single = np.array([np.load(filepath)])
            true_diags = get_true_diagonals(qubits, input_F, err)
            
            # Calculate all metrics for the loaded data
            delta_norms_2, delta_fidelities, delta_norms_1 = calculate_metrics_from_diags(diags_data_single, true_diags)

            # testing the second element
            estimated_second_element = diags_data_single[:, 1]
            
            # Delta b=1
            delta_b_1 = np.abs(np.array([true_diags[1] for _ in estimated_second_element]) - estimated_second_element)
            
            # Append one record per repeat
            all_data.append({
                "prefix": file_prefix if df_prefix is None else df_prefix,
                "qubits": int(qubits),
                "input_fidelity": input_F,
                "error_model": err,
                "total_shots": int(shots),
                "repeats": repeat_idx,
                "diag_sanity": np.sum(diags_data_single), # sums the vector; it should sum to 1
                "vector_a_norm_1": np.linalg.norm(diags_data_single, ord=1),
                "vector_a_norm_2": np.linalg.norm(diags_data_single),
                "delta_a_norm_1": delta_norms_1,
                "delta_a_norm_2": delta_norms_2,
                "est_fidelity": diags_data_single[0],
                "delta_fidelity": delta_fidelities,
                "delta_b=1": delta_b_1,
            })
                
    if not all_data:
        return pd.DataFrame(), pd.DataFrame()
        
    df = pd.DataFrame(all_data)
    
    return df

def load_bsqn_fidelity_estimation_data(output_dir: str):
    """
    Loads all 'bell_fidelity' data, calculates metrics, and returns a DataFrame.
    """
    all_data = []
    file_pattern = re.compile(
        r"bell_fidelity_(\d+)q_F([\d.]+)_err_(.*?)_shots_(\d+)_numstab_(\d+).npy"
    )
    
    if not os.path.isdir(output_dir):
        print(f"Warning: Fidelity data directory '{output_dir}' not found. Returning empty DataFrame.")
        return pd.DataFrame()

    for filename in os.listdir(output_dir):
        match = file_pattern.match(filename)
        if match:
            qubits, F_str, err, shots, numstab = match.groups()
            qubits = int(qubits)
            input_F = float(F_str)
            shots = int(shots)
            numstab = int(numstab)
            
            filepath = os.path.join(output_dir, filename)
            fidelities_array = np.load(filepath)
            
            for i, est_fidelity in enumerate(fidelities_array):
                all_data.append({
                    "qubits": qubits,
                    "input_fidelity": input_F,
                    "error_model": err,
                    "shots": shots,
                    "numstab": numstab,
                    "repeat": i,
                    "estimated_fidelity": est_fidelity,
                    "fidelity_error": np.abs(est_fidelity - input_F) # (est - true)
                })
    return pd.DataFrame(all_data)

In [ ]:
# Create zip file for experiment 1 and 2 for plotting for 
# "Performance comparison between BSQN and DGE for an 8-node complete graph state"

dge_df = load_multiruns_diagonal_data('data-new/depolarizing/dge_group_overlapped', 'tomo', 'DGE')
bsqn_df = load_multiruns_diagonal_data('data-new/depolarizing/bsqn', 'bell_diag', 'BSQN')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
dge_df.to_csv(f'{output_dir}/performance_analysis_dge_overlapped.zip', encoding='utf-8', index=False)
bsqn_df.to_csv(f'{output_dir}/performance_analysis_bsqn.zip', encoding='utf-8', index=False)

In [ ]:
# Create zip file for scalability experiment
# "Performance comparison between BSQN and DGE for an 2 to 200 nodes"

dge_df = load_multiruns_diagonal_data('data-new/scalability/dge_group_overlapped', 'tomo', 'DGE')
bsqn_df = load_scalability_data('data-new/scalability/bsqn', 'bell_scalability', 'BSQN')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
dge_df.to_csv(f'{output_dir}/scalability_analysis_dge_overlapped.zip', encoding='utf-8', index=False)
bsqn_df.to_csv(f'{output_dir}/scalability_analysis_bsqn.zip', encoding='utf-8', index=False)

In [18]:
# Create zip file for
# Fidelity estimation data via random sampling with BSQN

df = load_bsqn_fidelity_estimation_data('numerical_data/bell_fidelity_data')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
df.to_csv(f'{output_dir}/random_sampling_fidelity_estimation_bsqn.zip', encoding='utf-8', index=False)